In [1]:
from quantize_train import *

In [2]:
device = "cuda:0"
model = Model("models/yolov3-tiny.yaml", ch=3, nc=20).to(device)  # create

In [3]:
model = torch2quant(model)

Start convert pytorch to quantize, QCONFIG is:
[{'activation': functools.partial(<class 'qqquantize.observers.fake_quantize.FakeQuantize'>, observer=<class 'qqquantize.observers.minmaxobserver.MovingAverageMinMaxObserver'>, quantize_func=<class 'qqquantize.observers.fake_quantize.Fake_quantize_per_tensor'>, bits=16), 'weight': functools.partial(<class 'qqquantize.observers.fake_quantize.FakeQuantize'>, observer=<class 'qqquantize.observers.minmaxobserver.MinMaxObserver'>, quantize_func=<class 'qqquantize.observers.fake_quantize.Fake_quantize_per_tensor'>, bits=16), 'bias': functools.partial(<class 'qqquantize.observers.fake_quantize.FakeQuantize'>, bits=16)}]
QMAP is:
[{<class 'torch.nn.modules.linear.Linear'>: <class 'qqquantize.qmodules.qlinear.QLinear'>, <class 'torch.nn.modules.conv.Conv2d'>: <class 'qqquantize.qmodules.qconv.QConv2d'>, <class 'torch.nn.modules.conv.ConvTranspose2d'>: <class 'qqquantize.qmodules.qconvtranspose.QConvTranspose2d'>, <class 'torch.nn.modules.batchnorm.

In [4]:
model.eval()
disable_fake_quant(model)
enable_observer(model)

inp = torch.rand(1, 3, 416, 416).to(device)
_ = model(inp)

calc_qparams(model)
# swap_minmax_to_hist(model)
enable_observer(model)
enable_fake_quant(model)

In [5]:
for name, mod in model.named_modules():
    if isinstance(mod, qm.QConv2d):
        print(mod.weight_quant.scale)

tensor([7.62939e-06], device='cuda:0')
tensor([3.81470e-06], device='cuda:0')
tensor([1.90735e-06], device='cuda:0')
tensor([1.90735e-06], device='cuda:0')
tensor([9.53674e-07], device='cuda:0')
tensor([9.53674e-07], device='cuda:0')
tensor([4.76837e-07], device='cuda:0')
tensor([9.53674e-07], device='cuda:0')
tensor([9.53674e-07], device='cuda:0')
tensor([1.90735e-06], device='cuda:0')
tensor([9.53674e-07], device='cuda:0')
tensor([1.90735e-06], device='cuda:0')
tensor([1.90735e-06], device='cuda:0')


In [6]:
for name, mod in model.named_modules():
    if isinstance(mod, qm.QConv2d):
        mod.weight_quant.scale.fill_(3.1415926)

In [7]:
for name, mod in model.named_modules():
    if isinstance(mod, qm.QConv2d):
        print(mod.weight_quant.scale)

tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')
tensor([3.14159], device='cuda:0')


In [57]:
from quantize_utils import load_yolo_from_q_ckpt

In [80]:
model = load_yolo_from_q_ckpt("runs/train/q_maTact_mmTwt_concatfix_16bit_IP2_adjust1417/weights/best.pt", cfg="models/yolov3-tiny.yaml")

Start convert pytorch to quantize, QCONFIG is:
[{'activation': functools.partial(<class 'qqquantize.observers.fake_quantize.FakeQuantize'>, observer=<class 'qqquantize.observers.minmaxobserver.MovingAverageMinMaxObserver'>, quantize_func=<class 'qqquantize.observers.fake_quantize.Fake_quantize_per_tensor'>, bits=16), 'weight': functools.partial(<class 'qqquantize.observers.fake_quantize.FakeQuantize'>, observer=<class 'qqquantize.observers.minmaxobserver.MinMaxObserver'>, quantize_func=<class 'qqquantize.observers.fake_quantize.Fake_quantize_per_tensor'>, bits=16), 'bias': functools.partial(<class 'qqquantize.observers.fake_quantize.FakeQuantize'>, bits=16)}]
QMAP is:
[{<class 'torch.nn.modules.linear.Linear'>: <class 'qqquantize.qmodules.qlinear.QLinear'>, <class 'torch.nn.modules.conv.Conv2d'>: <class 'qqquantize.qmodules.qconv.QConv2d'>, <class 'torch.nn.modules.conv.ConvTranspose2d'>: <class 'qqquantize.qmodules.qconvtranspose.QConvTranspose2d'>, <class 'torch.nn.modules.batchnorm.

In [81]:
model.state_dict()['module.model.14.conv.weight_quant.scale']

tensor([3.05176e-05], device='cuda:0')

In [82]:
import torch

torch.log2(torch.tensor([0.01562]))

tensor([-6.00046])

In [83]:
for k in model.state_dict().keys():
    if "module.model.14.conv" in k:
        print(k)

module.model.14.conv.weight
module.model.14.conv.act_quant.fake_quant_enabled
module.model.14.conv.act_quant.observer_enabled
module.model.14.conv.act_quant.calc_qparams
module.model.14.conv.act_quant.scale
module.model.14.conv.act_quant.zero_point
module.model.14.conv.act_quant.observer.bits
module.model.14.conv.act_quant.observer.max_factor
module.model.14.conv.act_quant.observer.min_val
module.model.14.conv.act_quant.observer.max_val
module.model.14.conv.weight_quant.fake_quant_enabled
module.model.14.conv.weight_quant.observer_enabled
module.model.14.conv.weight_quant.calc_qparams
module.model.14.conv.weight_quant.scale
module.model.14.conv.weight_quant.zero_point
module.model.14.conv.weight_quant.observer.bits
module.model.14.conv.weight_quant.observer.max_factor
module.model.14.conv.weight_quant.observer.min_val
module.model.14.conv.weight_quant.observer.max_val


In [95]:
def ppp(state, conv_name):
#     wt = model.state_dict()['module.model.8.conv.weight'] 
#     sc = model.state_dict()['module.model.8.conv.weight_quant.scale']
    wt = state[conv_name]
    sc = state[conv_name + "_quant.scale"]
    miv = state[conv_name + "_quant.observer.min_val"]
    mav = state[conv_name + "_quant.observer.max_val"]
    
    print(miv)
    print(mav)
    print(state[conv_name + "_quant.observer.bits"])
    print(state[conv_name + "_quant.observer.max_factor"])
    
    
    print(torch.min(wt))
    print(torch.max(wt))
    print(sc, torch.log2(sc))

    qwt = torch.floor(wt / sc)
    print(torch.min(qwt))
    print(torch.max(qwt))
    print("#-"*40)

In [96]:
for k in model.state_dict().keys():
#     print(k)
    if k.endswith(".conv.weight"):
        print(k)
        ppp(model.state_dict(), k)

module.model.0.conv.weight
tensor(-1.23662, device='cuda:0')
tensor(1.14967, device='cuda:0')
tensor([16], device='cuda:0', dtype=torch.int32)
tensor([1.], device='cuda:0')
tensor(-1.23556, device='cuda:0')
tensor(1.14859, device='cuda:0')
tensor([6.10352e-05], device='cuda:0') tensor([-14.], device='cuda:0')
tensor(-20244., device='cuda:0')
tensor(18818., device='cuda:0')
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
module.model.2.conv.weight
tensor(-1.41611, device='cuda:0')
tensor(0.97086, device='cuda:0')
tensor([16], device='cuda:0', dtype=torch.int32)
tensor([1.], device='cuda:0')
tensor(-1.41588, device='cuda:0')
tensor(0.96995, device='cuda:0')
tensor([6.10352e-05], device='cuda:0') tensor([-14.], device='cuda:0')
tensor(-23198., device='cuda:0')
tensor(15891., device='cuda:0')
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
module.model.4.conv.weight
tensor(-0.58740, device='cuda:0')
tensor(0.76104, device='c

In [86]:
2 ** 15

32768